In [1]:
import numpy as np
import xarray as xr

In [2]:
nancy = xr.open_dataset("../netCDF_files/nancy_obs4_original.nc", decode_times=False)

In [3]:
nancy

<xarray.Dataset>
Dimensions:           (ObsIndex: 330158, ObsTypes: 45, copy: 166, locdim: 3, nlines: 576, qc_copy: 2)
Coordinates:
  * copy              (copy) int32 1 2 3 4 5 6 7 ... 160 161 162 163 164 165 166
  * qc_copy           (qc_copy) int32 1 2
  * ObsTypes          (ObsTypes) int32 1 2 3 4 5 6 7 8 ... 39 40 41 42 43 44 45
  * ObsIndex          (ObsIndex) int32 1 2 3 4 5 ... 330155 330156 330157 330158
Dimensions without coordinates: locdim, nlines
Data variables:
    ObsTypesMetaData  (ObsTypes) |S32 ...
    QCMetaData        (qc_copy) |S32 ...
    CopyMetaData      (copy) |S32 ...
    namelist          (nlines) |S256 ...
    time              (ObsIndex) float64 ...
    obs_type          (ObsIndex) int32 ...
    obs_keys          (ObsIndex) int32 ...
    observations      (ObsIndex, copy) float64 ...
    qc                (ObsIndex, qc_copy) int32 ...
    location          (ObsIndex, locdim) float64 ...
    which_vert        (ObsIndex) int32 ...
Attributes:
    creation_date

In [4]:
nancy['obs_type'].values

array([23, 21, 22, ..., 43, 25, 26], dtype=int32)

In [5]:
OBS_DIC = {
    1:"water_table_depth" ,
    2:"soil_temperature",
    3:"soil_moisture",
    4:"layer_liquid_water",
    5:"layer_ice",
    6:"snow_thickness",
    7:"snow_water",
    8:"modis_snowcover_frac",
    9:"modis_leaf_area_index",
    10:"modis_fpar",
    11:"biomass",
    12:"leaf_carbon",
    13:"leaf_area_index",
    14:"leaf_nitrogen",
    15:"tower_air_temperature",
    16:"tree_ring_npp_flux",
    17:"tower_soil_temperature",
    18:"tower_u_wind_component",
    19:"tower_v_wind_component",
    20:"tower_global_radiation",
    21:"tower_net_carbon_flux",
    22:"tower_latent_heat_flux",
    23:"tower_sensible_heat_flux",
    24:"tower_netc_eco_exchange",
    25:"tower_gpp_flux",
    26:"tower_er_flux",
    27:"soil_respiration_flux",
    28:"surface_albedo",
    29:"cosmos_neutron_intensity"
}

In [6]:
AVAILABLE_TYPES = []
for i in range(1,30,1):
    if i in nancy['obs_type'].values:
        AVAILABLE_TYPES.append(i)
print(AVAILABLE_TYPES)

[4, 6, 7, 10, 11, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28]


In [37]:
new = xr.open_dataset("../netCDF_files/template.nc", decode_times=False)

In [38]:
new

<xarray.Dataset>
Dimensions:         (obs: 330158, qc_copy: 2)
Coordinates:
    time            (obs) float64 ...
    lon             (obs) float64 ...
    lat             (obs) float64 ...
    vertical        (obs) float64 ...
  * qc_copy         (qc_copy) int32 -2147483647 -2147483647
Dimensions without coordinates: obs
Data variables:
    vertical_type   (obs) float64 ...
    observation     (obs) float64 ...
    error_variance  (obs) float64 ...
    qc              (obs, qc_copy) int32 ...
    obs_key         (obs) int32 ...
Attributes:
    featureType:  point
    title:        Restructured netCDF file from Nancy
    author:       Jason Ngo, SIParCS Intern
    source:       Nancy Collins, NCAR|UCAR
    conventions:  CF-1.7

In [9]:
searchval = 4
index_list = np.where(nancy['obs_type'].values == searchval)[0]

In [42]:
for searchval in AVAILABLE_TYPES:
    new = xr.open_dataset("../netCDF_files/template.nc", decode_times=False)
    index_list = np.where(nancy['obs_type'].values == searchval)[0] # index of observations of the same type
    j = 0
    location_matrix = nancy['location'].values.T
    
    for i in index_list:    
        new['lat'].values[i] = location_matrix[1][i]
    new = new.where( (np.isnan(new.coords['lat']) == False), drop=True)
    
    j = 0
    for i in index_list:
        new['observation'].values[j] = nancy['observations'].values.T[0][i]
        new['lon'].values[j] = location_matrix[0][i]
        new['vertical'].values[j] = location_matrix[2][i]
        new['time'].values[j] = nancy['time'].values[i]
        new['vertical_type'].values[j] = nancy['which_vert'].values[i]
        j+=1
    new.to_netcdf("{}.nc".format(OBS_DIC[searchval]))

In [40]:
AVAILABLE_TYPES

[4, 6, 7, 10, 11, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28]

In [19]:
test_np.drop_nan

AttributeError: 'numpy.ndarray' object has no attribute 'drop_nan'

In [24]:
test_np = test_np[np.logical_not(np.isnan(test_np))]

In [32]:
new = new.where( (np.isnan(new.coords['lat']) == False), drop=True)

In [33]:
new

<xarray.Dataset>
Dimensions:         (obs: 215394, qc_copy: 2)
Coordinates:
    time            (obs) float64 1.496e+05 1.496e+05 ... 1.496e+05 1.496e+05
    lon             (obs) float64 327.9 327.9 327.9 184.9 ... 227.6 227.6 227.6
    lat             (obs) float64 23.44 23.44 23.44 -57.19 ... 54.4 54.4 54.4
    vertical        (obs) float64 2.874e+04 2.874e+04 2.874e+04 ... 0.0 0.0 0.0
  * qc_copy         (qc_copy) int32 -2147483647 -2147483647
Dimensions without coordinates: obs
Data variables:
    vertical_type   (obs) float64 2.0 2.0 2.0 3.0 3.0 ... -1.0 -1.0 -1.0 -1.0
    observation     (obs) float64 239.2 9.4 2.7 264.3 ... 9.7 284.4 3.1 -2.6
    error_variance  (obs) float64 nan nan nan nan nan ... nan nan nan nan nan
    qc              (obs, qc_copy) float64 -2.147e+09 -2.147e+09 ... -2.147e+09
    obs_key         (obs) float64 -2.147e+09 -2.147e+09 ... -2.147e+09
Attributes:
    featureType:  point
    title:        Restructured netCDF file from Nancy
    author:       Jaso

In [ ]:
1+1